# FEBRERO 2022

In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format
import json
import csv

In [2]:
start_date = '2022-02-01'
final_date = '2022-02-28'

### PARAMETRIA COMISIONES

In [3]:
parametria_comisiones = pd.read_csv(r'G:\My Drive\RESPALDO-PC\TOPUPS\PARAMETRIA TOPUPS.csv',dtype={'monto_recarga':float})
parametria_comisiones['inicio'] = pd.to_datetime(parametria_comisiones['inicio'])
parametria_comisiones['porcentaje_comision_final'] = parametria_comisiones['porcentaje_comision']/100
parametria_comisiones['index_comisiones'] = (parametria_comisiones['carrier'].astype(str).replace(' ', '',regex=True)+'_'
                                             +parametria_comisiones['monto_recarga'].astype(str))
parametria_comisiones.head()

,motivo_del_cambio,naturaleza,estatus,inicio,fin,carrier,monto_recarga,porcentaje_comision,categoria,porcentaje_comision_final,index_comisiones
0,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,30.00,7.00,MOVISTAR,0.07,MOVISTAR_30.0
1,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,50.00,7.00,MOVISTAR,0.07,MOVISTAR_50.0
2,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,100.00,7.00,MOVISTAR,0.07,MOVISTAR_100.0
3,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,200.00,7.00,MOVISTAR,0.07,MOVISTAR_200.0
4,NO APLICA,FIJA,VIGENTE,2020-01-08,2999-12-31,MOVISTAR,500.00,7.00,MOVISTAR,0.07,MOVISTAR_500.0


In [4]:
parametria_comisiones.shape

(26, 11)

In [5]:
parametria_comisiones.dtypes

motivo_del_cambio                    object
naturaleza                           object
estatus                              object
inicio                       datetime64[ns]
fin                                  object
carrier                              object
monto_recarga                       float64
porcentaje_comision                 float64
categoria                            object
porcentaje_comision_final           float64
index_comisiones                     object
dtype: object

In [6]:
parametria_comisiones['carrier'].unique()

array(['MOVISTAR', 'UNEFON', 'TELCEL', 'ATT', 'IUSACELL'], dtype=object)

In [7]:
parametria_comisiones['index_comisiones'].unique()

array(['MOVISTAR_30.0', 'MOVISTAR_50.0', 'MOVISTAR_100.0',
       'MOVISTAR_200.0', 'MOVISTAR_500.0', 'UNEFON_30.0', 'UNEFON_50.0',
       'UNEFON_100.0', 'UNEFON_200.0', 'UNEFON_500.0', 'TELCEL_30.0',
       'TELCEL_50.0', 'TELCEL_80.0', 'TELCEL_100.0', 'TELCEL_200.0',
       'TELCEL_500.0', 'ATT_30.0', 'ATT_50.0', 'ATT_100.0', 'ATT_200.0',
       'ATT_500.0', 'IUSACELL_30.0', 'IUSACELL_50.0', 'IUSACELL_100.0',
       'IUSACELL_200.0', 'IUSACELL_500.0'], dtype=object)

### SERTI (EXTRAIDO EN EXCEL)

In [8]:
serti_files = glob.glob(r'G:\My Drive\RESPALDO-PC\TOPUPS\202*\**\SERTI - COMISIONES\*.csv',recursive=False)
serti_files


['G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\12. DICIEMBRE\\SERTI - COMISIONES\\0000000000002610_2021-12-29.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\12. DICIEMBRE\\SERTI - COMISIONES\\0000000000002610_2021-12-30.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\12. DICIEMBRE\\SERTI - COMISIONES\\0000000000002610_2021-12-31.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\SERTI - COMISIONES\\0000000000002610_2022-01-01.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\SERTI - COMISIONES\\0000000000002610_2022-01-02.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\SERTI - COMISIONES\\0000000000002610_2022-01-03.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\SERTI - COMISIONES\\0000000000002610_2022-01-04.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\SERTI - COMISIONES\\0000000000002610_2022-01-05.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\SERTI - COMISIONES\\0000000000002610_2022-01-06.csv',
 'G:\\My Drive\\

In [9]:
serti = pd.concat([pd.read_csv(f,dtype={'bin':object,'panTerminacion':object,'claveRechazo':object,'phone':object
                                        ,'transactionId':object,'importe':float})
                   .assign(file_name=os.path.basename(f)) for f in serti_files])
serti.loc[serti['carrier'].str.contains('AT&T'), 'carrier'] = 'ATT'
serti['carrier'] = serti['carrier'].str.strip()
serti['carrier'] = serti['carrier'].str.upper()
serti['last4_phone_serti'] = serti['phone'].str[-4:]
serti['serti_date'] = serti['date'].str[:10]
serti['serti_date'] = pd.to_datetime(serti['serti_date'])
serti['comision'] = serti['comision']/100
serti['ivaComision'] = serti['ivaComision']/100
serti.sort_values(by=['date'],inplace=True)
serti['INDEX_TOTAL'] = (serti['transactionId'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['last4_phone_serti'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['panTerminacion'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['importe'].astype(str).replace(' ', '',regex=True))
serti['index_comisiones'] = (serti['carrier'].astype(str).replace(' ', '',regex=True) + '_'
                       + serti['importe'].astype(str).replace(' ', '',regex=True))
serti.reset_index(inplace=True)
serti.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,comision,ivaComision,carrier,phone,folio,file_name,last4_phone_serti,serti_date,INDEX_TOTAL,index_comisiones
0,0,557907,9724,100.00,2021-12-28 18:00:22.455,49679728,Chip,002202,NaN,0,...,7.00,1.12,MOVISTAR,******4628,"5,918.00",0000000000002610_2021-12-29.csv,4628,2021-12-28,49679728_4628_9724_100.0,MOVISTAR_100.0
1,1,525678,1991,50.00,2021-12-28 18:01:27.719,49679736,Chip,NaN,NaN,0,...,0.00,0.00,TELCEL,******2216,NaN,0000000000002610_2021-12-29.csv,2216,2021-12-28,49679736_2216_1991_50.0,TELCEL_50.0
2,2,402766,7023,50.00,2021-12-28 18:02:36.545,49679738,Chip,033735,NaN,0,...,2.00,0.32,TELCEL,******3872,"430,625.00",0000000000002610_2021-12-29.csv,3872,2021-12-28,49679738_3872_7023_50.0,TELCEL_50.0
3,3,415231,4828,50.00,2021-12-28 18:03:09.449,49679744,Chip,368181,NaN,0,...,2.00,0.32,TELCEL,******9736,"483,936.00",0000000000002610_2021-12-29.csv,9736,2021-12-28,49679744_9736_4828_50.0,TELCEL_50.0
4,4,525678,6792,100.00,2021-12-28 18:03:13.909,49679745,Chip,000621,NaN,0,...,4.00,0.64,TELCEL,******9901,"903,006.00",0000000000002610_2021-12-29.csv,9901,2021-12-28,49679745_9901_6792_100.0,TELCEL_100.0


In [10]:
serti.shape

(606698, 25)

In [11]:
serti.columns

Index(['index', 'bin', 'panTerminacion', 'importe', 'date', 'transactionId',
       'entryMode', 'numAuth', 'terminal', 'caja', 'aprobada', 'claveRechazo',
       'merchant', 'producto', 'tarjetahabiente', 'comision', 'ivaComision',
       'carrier', 'phone', 'folio', 'file_name', 'last4_phone_serti',
       'serti_date', 'INDEX_TOTAL', 'index_comisiones'],
      dtype='object')

In [12]:
serti['index_comisiones'].unique()

array(['MOVISTAR_100.0', 'TELCEL_50.0', 'TELCEL_100.0', 'MOVISTAR_50.0',
       'UNEFON_50.0', 'UNEFON_30.0', 'TELCEL_200.0', 'ATT_30.0',
       'ATT_100.0', 'TELCEL_30.0', 'ATT_50.0', 'MOVISTAR_30.0',
       'TELCEL_80.0', 'ATT_200.0', 'MOVISTAR_200.0', 'UNEFON_100.0',
       'UNEFON_200.0', 'TELCEL_500.0', 'MOVISTAR_500.0', 'UNEFON_500.0',
       'ATT_500.0'], dtype=object)

In [13]:
duplicates_serti = serti[serti.duplicated(['transactionId'] , keep=False)]
duplicates_serti.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,comision,ivaComision,carrier,phone,folio,file_name,last4_phone_serti,serti_date,INDEX_TOTAL,index_comisiones


In [14]:
duplicates_serti.shape

(0, 25)

In [15]:
serti.drop_duplicates(subset='INDEX_TOTAL', keep='first', inplace=True)

In [16]:
serti.shape

(606698, 25)

In [17]:
serti.dtypes

index                         int64
bin                          object
panTerminacion               object
importe                     float64
date                         object
transactionId                object
entryMode                    object
numAuth                      object
terminal                    float64
caja                          int64
aprobada                       bool
claveRechazo                 object
merchant                      int64
producto                     object
tarjetahabiente              object
comision                    float64
ivaComision                 float64
carrier                      object
phone                        object
folio                       float64
file_name                    object
last4_phone_serti            object
serti_date           datetime64[ns]
INDEX_TOTAL                  object
index_comisiones             object
dtype: object

In [18]:
serti_period =  serti[(serti['serti_date'] >= start_date) & (serti['serti_date'] <= final_date)]
serti_period['serti_date'].unique()

array(['2022-02-01T00:00:00.000000000', '2022-02-02T00:00:00.000000000',
       '2022-02-03T00:00:00.000000000', '2022-02-04T00:00:00.000000000',
       '2022-02-05T00:00:00.000000000', '2022-02-06T00:00:00.000000000',
       '2022-02-07T00:00:00.000000000', '2022-02-08T00:00:00.000000000',
       '2022-02-09T00:00:00.000000000', '2022-02-10T00:00:00.000000000',
       '2022-02-11T00:00:00.000000000', '2022-02-12T00:00:00.000000000',
       '2022-02-13T00:00:00.000000000', '2022-02-14T00:00:00.000000000',
       '2022-02-15T00:00:00.000000000', '2022-02-16T00:00:00.000000000',
       '2022-02-17T00:00:00.000000000', '2022-02-18T00:00:00.000000000',
       '2022-02-19T00:00:00.000000000', '2022-02-20T00:00:00.000000000',
       '2022-02-21T00:00:00.000000000', '2022-02-22T00:00:00.000000000',
       '2022-02-23T00:00:00.000000000', '2022-02-24T00:00:00.000000000',
       '2022-02-25T00:00:00.000000000', '2022-02-26T00:00:00.000000000',
       '2022-02-27T00:00:00.000000000', '2022-02-28

### SERTI CON PARAMETRÍA DE COMISIONES

In [19]:
serti_final = serti_period.merge(parametria_comisiones[['index_comisiones','estatus','porcentaje_comision_final']]
                      ,on='index_comisiones',how='left')
serti_final['estatus'].fillna('Not_found',inplace=True)
serti_final['porcentaje_comision_final'].fillna(0,inplace=True)
serti_final['comision_db'] = serti_final['importe'] * serti_final['porcentaje_comision_final']
serti_final['iva_comision_db'] = serti_final['comision_db'] * 0.1600
serti_final.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,folio,file_name,last4_phone_serti,serti_date,INDEX_TOTAL,index_comisiones,estatus,porcentaje_comision_final,comision_db,iva_comision_db
0,126,416916,5006,30.00,2022-02-01 00:01:36.224,51581310,Chip,"359,997.00",NaN,0,...,"235,185.00",0000000000002610_2022-02-01.csv,8856,2022-02-01,51581310_8856_5006_30.0,TELCEL_30.0,VIGENTE,0.04,1.20,0.19
1,127,551238,5848,100.00,2022-02-01 00:02:04.603,51581316,Chip,"273,136.00",NaN,0,...,"125,977.00",0000000000002610_2022-02-01.csv,3197,2022-02-01,51581316_3197_5848_100.0,MOVISTAR_100.0,VIGENTE,0.07,7.00,1.12
2,128,506429,2954,100.00,2022-02-01 00:05:37.541,51581339,Chip,"539,954.00",NaN,0,...,"550,630.00",0000000000002610_2022-02-01.csv,7206,2022-02-01,51581339_7206_2954_100.0,TELCEL_100.0,VIGENTE,0.04,4.00,0.64
3,129,491566,4547,100.00,2022-02-01 00:08:16.416,51581364,Chip,153.00,NaN,0,...,"258,749.00",0000000000002610_2022-02-01.csv,3879,2022-02-01,51581364_3879_4547_100.0,ATT_100.0,VIGENTE,0.07,7.00,1.12
4,130,417849,2645,50.00,2022-02-01 00:08:37.568,51581367,Chip,"18,556.00",NaN,0,...,"659,527.00",0000000000002610_2022-02-01.csv,9204,2022-02-01,51581367_9204_2645_50.0,TELCEL_50.0,VIGENTE,0.04,2.00,0.32


In [20]:
serti_final.shape

(255764, 29)

### PIVOT SERTI

In [21]:
pivot_serti = pd.pivot_table(serti_final,index=['serti_date']
                            ,columns=['aprobada']
                            ,values=['importe','transactionId','comision','ivaComision','comision_db','iva_comision_db']
                            ,aggfunc={'importe':np.sum,
                                     'transactionId':len,
                                      'comision':np.sum,
                                      'ivaComision':np.sum,
                                     'comision_db':np.sum,
                                     'iva_comision_db':np.sum,},margins=True).reset_index()
pivot_serti.fillna(0,inplace=True)
pivot_serti.head()

serti_date comision                     comision_db  \
aprobada                         False      True       All       False   
0         2022-02-01 00:00:00     0.00 29,407.00 29,407.00    3,460.40   
1         2022-02-02 00:00:00     0.00 25,783.90 25,783.90    2,336.80   
2         2022-02-03 00:00:00     0.00 26,786.30 26,786.30    2,566.00   
3         2022-02-04 00:00:00     0.00 28,377.00 28,377.00    2,511.30   
4         2022-02-05 00:00:00     0.00 27,424.50 27,424.50    2,296.10   

                               importe                       ivaComision  \
aprobada      True       All     False       True        All       False   
0        29,407.00 32,867.40 66,320.00 571,960.00 638,280.00        0.00   
1        25,783.90 28,120.70 45,010.00 503,590.00 548,600.00        0.00   
2        26,786.30 29,352.30 50,470.00 523,070.00 573,540.00        0.00   
3        28,377.00 30,888.30 47,370.00 553,830.00 601,200.00        0.00   
4        27,424.50 29,720.60 44,690.00 532,950.00 577,640.00        0.00   

                           iva_comision_db                   transactionId  \
aprobada     True      All           False     True      All         False   
0        4,705.12 4,705.12          553.66 4,705.12 5,258.78           970   
1        4,125.42 4,125.42          373.89 4,125.42 4,499.31           716   
2        4,285.81 4,285.81          410.56 4,285.81 4,696.37           743   
3        4,540.32 4,540.32          401.81 4,540.32 4,942.13           700   
4        4,387.92 4,387.92          367.38 4,387.92 4,755.30           719   

                      
aprobada  True   All  
0         8567  9537  
1         7736  8452  
2         7857  8600  
3         8298  8998  
4         8323  9042

In [22]:
pivot_serti.shape

(29, 19)

### DB PRODUCTO

In [23]:
producto_files = glob.glob(r'G:\My Drive\RESPALDO-PC\TOPUPS\202*\**\DB\TOPUPS*.csv',recursive=False)
producto_files


['G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\05. MAYO\\DB\\TOPUPS MAYO 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\06. JUNIO\\DB\\TOPUPS JUNIO 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\08. AGOSTO\\DB\\TOPUPS AGOSTO 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\12. DICIEMBRE\\DB\\TOPUPS DICIEMBRE 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\11. NOVIEMBRE\\DB\\TOPUPS OCTUBRE - NOVIEMBRE 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\07. JULIO\\DB\\TOPUPS JULIO 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2021\\09. SEPTIEMBRE\\DB\\TOPUPS SEPTIEMBRE 2021.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\01. ENERO\\DB\\TOPUPS ENERO 2022.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\02. FEBRERO\\DB\\TOPUPS FEBRERO 2022.csv',
 'G:\\My Drive\\RESPALDO-PC\\TOPUPS\\2022\\03. MARZO\\DB\\TOPUPS MARZO 2022.csv']

In [24]:
topups_db = pd.concat([pd.read_csv(f,dtype={'clip_external_tracking_id':object,'serti_transaction_id':object,
                                            'phone_number':object,'last4':object,'amount':float,'status_code':object})
                   .assign(file_name=os.path.basename(f)) for f in producto_files])
topups_db['clip_external_tracking_id'].fillna('sin_order_id_db',inplace=True)
topups_db['serti_transaction_id'].fillna('sin_order_id_db',inplace=True)
topups_db['created_at'] = pd.to_datetime(topups_db['created_at'])
topups_db['updated_at'] = pd.to_datetime(topups_db['updated_at'])
topups_db['last4_phone'] = topups_db['phone_number'].str[-4:]
topups_db['INDEX_TOTAL'] = (topups_db['serti_transaction_id'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4_phone'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['last4'].astype(str).replace(' ', '',regex=True) + '_'
                           + topups_db['amount'].astype(str).replace(' ', '',regex=True))
topups_db['index_comisiones'] = (topups_db['carrier'].astype(str).replace(' ', '',regex=True)+'-'
                                             +topups_db['amount'].astype(str))
topups_db.head()

C:\Users\CRISTI~1.APA\AppData\Local\Temp/ipykernel_13992/1884929116.py:1: DtypeWarning: Columns (15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  topups_db = pd.concat([pd.read_csv(f,dtype={'clip_external_tracking_id':object,'serti_transaction_id':object,
C:\Users\CRISTI~1.APA\AppData\Local\Temp/ipykernel_13992/1884929116.py:1: DtypeWarning: Columns (15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  topups_db = pd.concat([pd.read_csv(f,dtype={'clip_external_tracking_id':object,'serti_transaction_id':object,
C:\Users\CRISTI~1.APA\AppData\Local\Temp/ipykernel_13992/1884929116.py:1: DtypeWarning: Columns (15,19) have mixed types. Specify dtype option on import or set low_memory=False.
  topups_db = pd.concat([pd.read_csv(f,dtype={'clip_external_tracking_id':object,'serti_transaction_id':object,
C:\Users\CRISTI~1.APA\AppData\Local\Temp/ipykernel_13992/1884929116.py:1: DtypeWarning: Columns (15,19) have mixed types. Specify 

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier,phone_number,date_created,created_at,payment_type,status_code,...,file_name,date_created_mexico_city,created_at_mexico_city,updated_at,created_at_datetime,updated_at_datetime,serti_transaction_id,last4_phone,INDEX_TOTAL,index_comisiones
0,588f8153-01f9-4ccd-ae99-34be495f3caa,e0d19424-484f-4a22-97ff-bce7d79f0c63,ff80481a-afb6-452c-9821-451268396ced,34205977,UNEFON,5621454162,2021-05-01,2021-05-01 05:00:20,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,4162,sin_order_id_db_4162_7364_50.0,UNEFON-50.0
1,e8b3d057-dc8d-4f0c-8687-754a1c7b84a5,422b5ffc-82ed-4b33-8374-477915a34574,54c6e184-d9a6-43e3-820b-be8c983d1aac,34206011,TELCEL,5510080854,2021-05-01,2021-05-01 05:01:22,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,0854,sin_order_id_db_0854_5366_100.0,TELCEL-100.0
2,33e06d50-9718-42dd-b213-96a923414b1f,3164f6fc-dbf2-49b6-b7d3-3c20576c0d10,672af042-e82d-42d9-b566-ad6660633592,34206035,TELCEL,6648088827,2021-05-01,2021-05-01 05:02:19,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,8827,sin_order_id_db_8827_7946_100.0,TELCEL-100.0
3,12c27a23-a6ad-4a59-94ba-2611fd04aa8c,c3e69929-3222-4875-8f7f-8f9204546f49,502276d4-964d-4dd0-8a9b-6677c0298eef,34206062,TELCEL,4494897918,2021-05-01,2021-05-01 05:03:06,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,7918,sin_order_id_db_7918_2697_50.0,TELCEL-50.0
4,54d9a6e8-9b43-4084-a141-3706985723b7,4581385b-e1e3-4642-8355-8cb2fc25db35,1875f768-b7fd-4623-bd48-401157b2c70e,34206091,ATT,3320491258,2021-05-01,2021-05-01 05:04:10,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,1258,sin_order_id_db_1258_3500_50.0,ATT-50.0


In [25]:
topups_db.shape

(2361865, 31)

In [26]:
topups_db['carrier'].unique()

array(['UNEFON', 'TELCEL', 'ATT', 'MOVISTAR'], dtype=object)

In [27]:
duplicates = topups_db[topups_db.duplicated(['serti_transaction_id'], keep=False)]
duplicates.head()

,merchant_id,topup_id,transaction_id,clip_external_tracking_id,carrier,phone_number,date_created,created_at,payment_type,status_code,...,file_name,date_created_mexico_city,created_at_mexico_city,updated_at,created_at_datetime,updated_at_datetime,serti_transaction_id,last4_phone,INDEX_TOTAL,index_comisiones
0,588f8153-01f9-4ccd-ae99-34be495f3caa,e0d19424-484f-4a22-97ff-bce7d79f0c63,ff80481a-afb6-452c-9821-451268396ced,34205977,UNEFON,5621454162,2021-05-01,2021-05-01 05:00:20,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,4162,sin_order_id_db_4162_7364_50.0,UNEFON-50.0
1,e8b3d057-dc8d-4f0c-8687-754a1c7b84a5,422b5ffc-82ed-4b33-8374-477915a34574,54c6e184-d9a6-43e3-820b-be8c983d1aac,34206011,TELCEL,5510080854,2021-05-01,2021-05-01 05:01:22,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,0854,sin_order_id_db_0854_5366_100.0,TELCEL-100.0
2,33e06d50-9718-42dd-b213-96a923414b1f,3164f6fc-dbf2-49b6-b7d3-3c20576c0d10,672af042-e82d-42d9-b566-ad6660633592,34206035,TELCEL,6648088827,2021-05-01,2021-05-01 05:02:19,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,8827,sin_order_id_db_8827_7946_100.0,TELCEL-100.0
3,12c27a23-a6ad-4a59-94ba-2611fd04aa8c,c3e69929-3222-4875-8f7f-8f9204546f49,502276d4-964d-4dd0-8a9b-6677c0298eef,34206062,TELCEL,4494897918,2021-05-01,2021-05-01 05:03:06,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,7918,sin_order_id_db_7918_2697_50.0,TELCEL-50.0
4,54d9a6e8-9b43-4084-a141-3706985723b7,4581385b-e1e3-4642-8355-8cb2fc25db35,1875f768-b7fd-4623-bd48-401157b2c70e,34206091,ATT,3320491258,2021-05-01,2021-05-01 05:04:10,CARD,1,...,TOPUPS MAYO 2021.csv,NaN,NaN,NaT,NaN,NaN,sin_order_id_db,1258,sin_order_id_db_1258_3500_50.0,ATT-50.0


In [28]:
duplicates.shape

(1198162, 31)

In [29]:
duplicates['serti_transaction_id'].unique()

array(['sin_order_id_db', '0'], dtype=object)

In [30]:
topups_db.shape

(2361865, 31)

In [31]:
topups_db.dtypes

merchant_id                          object
topup_id                             object
transaction_id                       object
clip_external_tracking_id            object
carrier                              object
phone_number                         object
date_created                         object
created_at                   datetime64[ns]
payment_type                         object
status_code                          object
status_msg                           object
amount                              float64
processor                            object
issuer_auth_code                     object
folio_id                            float64
receipt_no                           object
last4                                object
authNum                              object
orderId                               int64
type                                 object
response_code                        object
file_name                            object
date_created_mexico_city        

### DB CON PARAMETRÍA DE COMISIONES

In [32]:
topups_db_final = topups_db

In [33]:
# topups_db_final = topups_db.merge(parametria_comisiones[['index_comisiones','estatus','porcentaje_comision_final']]
#                       ,on='index_comisiones',how='left')
# topups_db_final['estatus'].fillna('Not_found',inplace=True)
# topups_db_final['porcentaje_comision_final'].fillna(0,inplace=True)
# topups_db_final['comision_db'] = topups_db_final['amount'] * topups_db_final['porcentaje_comision_final']
# topups_db_final['iva_comision_db'] = topups_db_final['comision_db'] * 0.1600
# topups_db_final.head()

In [34]:
topups_db_final.shape

(2361865, 31)

### SERTI VS DB

In [35]:
cruce_1 = serti_final.merge(topups_db_final[['INDEX_TOTAL','topup_id','transaction_id','clip_external_tracking_id',
                                             'serti_transaction_id','created_at','updated_at','status_code',
                                             'status_msg','amount','last4_phone','carrier','type',
                                             'response_code']]
                      ,on='INDEX_TOTAL',how='inner')
cruce_1['status_code'].fillna('Not_found',inplace=True)
cruce_1['topup_id'].fillna('Not_found',inplace=True)
cruce_1.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,serti_transaction_id,created_at,updated_at,status_code,status_msg,amount,last4_phone,carrier_y,type,response_code
0,126,416916,5006,30.00,2022-02-01 00:01:36.224,51581310,Chip,"359,997.00",NaN,0,...,51581310,2022-02-01,2022-02-01,1,Approved,30.00,8856,TELCEL,emv_sale,00
1,127,551238,5848,100.00,2022-02-01 00:02:04.603,51581316,Chip,"273,136.00",NaN,0,...,51581316,2022-02-01,2022-02-01,1,Approved,100.00,3197,MOVISTAR,emv_sale,00
2,128,506429,2954,100.00,2022-02-01 00:05:37.541,51581339,Chip,"539,954.00",NaN,0,...,51581339,2022-02-01,2022-02-01,1,Approved,100.00,7206,TELCEL,emv_sale,00
3,129,491566,4547,100.00,2022-02-01 00:08:16.416,51581364,Chip,153.00,NaN,0,...,51581364,2022-02-01,2022-02-01,1,Approved,100.00,3879,ATT,emv_sale,00
4,130,417849,2645,50.00,2022-02-01 00:08:37.568,51581367,Chip,"18,556.00",NaN,0,...,51581367,2022-02-01,2022-02-01,1,Approved,50.00,9204,TELCEL,emv_sale,00


In [36]:
cruce_1.shape

(255601, 42)

In [37]:
cruce_1['aprobada'].unique()

array([ True, False])

### TRUE ONLY

In [38]:
cruce_1_true = cruce_1[(cruce_1['aprobada'] == True)]
cruce_1_true['amount'].fillna(0,inplace=True)
cruce_1_true.head()

C:\Users\CRISTI~1.APA\AppData\Local\Temp/ipykernel_13992/1833023460.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_true['amount'].fillna(0,inplace=True)


,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,serti_transaction_id,created_at,updated_at,status_code,status_msg,amount,last4_phone,carrier_y,type,response_code
0,126,416916,5006,30.00,2022-02-01 00:01:36.224,51581310,Chip,"359,997.00",NaN,0,...,51581310,2022-02-01,2022-02-01,1,Approved,30.00,8856,TELCEL,emv_sale,00
1,127,551238,5848,100.00,2022-02-01 00:02:04.603,51581316,Chip,"273,136.00",NaN,0,...,51581316,2022-02-01,2022-02-01,1,Approved,100.00,3197,MOVISTAR,emv_sale,00
2,128,506429,2954,100.00,2022-02-01 00:05:37.541,51581339,Chip,"539,954.00",NaN,0,...,51581339,2022-02-01,2022-02-01,1,Approved,100.00,7206,TELCEL,emv_sale,00
3,129,491566,4547,100.00,2022-02-01 00:08:16.416,51581364,Chip,153.00,NaN,0,...,51581364,2022-02-01,2022-02-01,1,Approved,100.00,3879,ATT,emv_sale,00
4,130,417849,2645,50.00,2022-02-01 00:08:37.568,51581367,Chip,"18,556.00",NaN,0,...,51581367,2022-02-01,2022-02-01,1,Approved,50.00,9204,TELCEL,emv_sale,00


In [39]:
cruce_1_true.shape

(232644, 42)

In [40]:
duplicates_cruce_1_true = cruce_1_true[cruce_1_true.duplicated(['INDEX_TOTAL'] , keep=False)]
duplicates_cruce_1_true.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,serti_transaction_id,created_at,updated_at,status_code,status_msg,amount,last4_phone,carrier_y,type,response_code


In [41]:
duplicates_cruce_1_true.shape

(0, 42)

In [42]:
cruce_1_final = cruce_1_true[(cruce_1_true['topup_id'] != 'Not_found')]
cruce_1_final.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,serti_transaction_id,created_at,updated_at,status_code,status_msg,amount,last4_phone,carrier_y,type,response_code
0,126,416916,5006,30.00,2022-02-01 00:01:36.224,51581310,Chip,"359,997.00",NaN,0,...,51581310,2022-02-01,2022-02-01,1,Approved,30.00,8856,TELCEL,emv_sale,00
1,127,551238,5848,100.00,2022-02-01 00:02:04.603,51581316,Chip,"273,136.00",NaN,0,...,51581316,2022-02-01,2022-02-01,1,Approved,100.00,3197,MOVISTAR,emv_sale,00
2,128,506429,2954,100.00,2022-02-01 00:05:37.541,51581339,Chip,"539,954.00",NaN,0,...,51581339,2022-02-01,2022-02-01,1,Approved,100.00,7206,TELCEL,emv_sale,00
3,129,491566,4547,100.00,2022-02-01 00:08:16.416,51581364,Chip,153.00,NaN,0,...,51581364,2022-02-01,2022-02-01,1,Approved,100.00,3879,ATT,emv_sale,00
4,130,417849,2645,50.00,2022-02-01 00:08:37.568,51581367,Chip,"18,556.00",NaN,0,...,51581367,2022-02-01,2022-02-01,1,Approved,50.00,9204,TELCEL,emv_sale,00


In [43]:
cruce_1_final.shape

(232644, 42)

In [44]:
na_cruce_1_final = cruce_1_true[(cruce_1_true['topup_id'] == 'Not_found')]
na_cruce_1_final.head()

,index,bin,panTerminacion,importe,date,transactionId,entryMode,numAuth,terminal,caja,...,serti_transaction_id,created_at,updated_at,status_code,status_msg,amount,last4_phone,carrier_y,type,response_code


In [45]:
na_cruce_1_final.shape

(0, 42)

In [46]:
cruce_1_true.columns

Index(['index', 'bin', 'panTerminacion', 'importe', 'date', 'transactionId',
       'entryMode', 'numAuth', 'terminal', 'caja', 'aprobada', 'claveRechazo',
       'merchant', 'producto', 'tarjetahabiente', 'comision', 'ivaComision',
       'carrier_x', 'phone', 'folio', 'file_name', 'last4_phone_serti',
       'serti_date', 'INDEX_TOTAL', 'index_comisiones', 'estatus',
       'porcentaje_comision_final', 'comision_db', 'iva_comision_db',
       'topup_id', 'transaction_id', 'clip_external_tracking_id',
       'serti_transaction_id', 'created_at', 'updated_at', 'status_code',
       'status_msg', 'amount', 'last4_phone', 'carrier_y', 'type',
       'response_code'],
      dtype='object')

In [47]:
cruce_1.dtypes

index                                 int64
bin                                  object
panTerminacion                       object
importe                             float64
date                                 object
transactionId                        object
entryMode                            object
numAuth                              object
terminal                            float64
caja                                  int64
aprobada                               bool
claveRechazo                         object
merchant                              int64
producto                             object
tarjetahabiente                      object
comision                            float64
ivaComision                         float64
carrier_x                            object
phone                                object
folio                               float64
file_name                            object
last4_phone_serti                    object
serti_date                   dat

### PIVOT TRUE ONLY

In [48]:
pivot_true = pd.pivot_table(cruce_1_true,index=['serti_date']
                            ,columns=['status_code']
                            ,values=['importe','amount','transactionId','serti_transaction_id','comision','ivaComision',
                                     'comision_db','iva_comision_db']
                            ,aggfunc={'importe':np.sum,
                                      'amount':np.sum,
                                      'comision':np.sum,
                                      'ivaComision':np.sum,
                                      'comision_db':np.sum,
                                      'iva_comision_db':np.sum,
                                      'transactionId':len,
                                      'serti_transaction_id':len},margins=True).reset_index(0)
pivot_true.fillna(0,inplace=True)
pivot_true.head()

serti_date     amount             comision            \
status_code                               1        All         1       All   
0            2022-02-01 00:00:00 571,910.00 571,910.00 29,403.50 29,403.50   
1            2022-02-02 00:00:00 503,590.00 503,590.00 25,783.90 25,783.90   
2            2022-02-03 00:00:00 523,070.00 523,070.00 26,786.30 26,786.30   
3            2022-02-04 00:00:00 553,830.00 553,830.00 28,377.00 28,377.00   
4            2022-02-05 00:00:00 532,820.00 532,820.00 27,415.40 27,415.40   

            comision_db              importe            ivaComision           \
status_code           1       All          1        All           1      All   
0             29,403.50 29,403.50 571,910.00 571,910.00    4,704.56 4,704.56   
1             25,783.90 25,783.90 503,590.00 503,590.00    4,125.42 4,125.42   
2             26,786.30 26,786.30 523,070.00 523,070.00    4,285.81 4,285.81   
3             28,377.00 28,377.00 553,830.00 553,830.00    4,540.32 4,540.32   
4             27,415.40 27,415.40 532,820.00 532,820.00    4,386.46 4,386.46   

            iva_comision_db          serti_transaction_id       transactionId  \
status_code               1      All                    1   All             1   
0                  4,704.56 4,704.56                 8566  8566          8566   
1                  4,125.42 4,125.42                 7736  7736          7736   
2                  4,285.81 4,285.81                 7857  7857          7857   
3                  4,540.32 4,540.32                 8298  8298          8298   
4                  4,386.46 4,386.46                 8321  8321          8321   

                   
status_code   All  
0            8566  
1            7736  
2            7857  
3            8298  
4            8321

In [49]:
pivot_true.shape

(29, 17)

In [50]:
stacked = pivot_true.stack(level=[1]).reset_index()
stacked.head()

,level_0,status_code,amount,comision,comision_db,importe,ivaComision,iva_comision_db,serti_date,serti_transaction_id,transactionId
0,0,,NaN,NaN,NaN,NaN,NaN,NaN,2022-02-01 00:00:00,NaN,NaN
1,0,1,"571,910.00","29,403.50","29,403.50","571,910.00","4,704.56","4,704.56",NaN,"8,566.00","8,566.00"
2,0,All,"571,910.00","29,403.50","29,403.50","571,910.00","4,704.56","4,704.56",NaN,"8,566.00","8,566.00"
3,1,,NaN,NaN,NaN,NaN,NaN,NaN,2022-02-02 00:00:00,NaN,NaN
4,1,1,"503,590.00","25,783.90","25,783.90","503,590.00","4,125.42","4,125.42",NaN,"7,736.00","7,736.00"


In [51]:
stacked.shape

(87, 11)

### PIVOT TRUE Y CARRIER 

In [52]:
pivot_carrier = pd.pivot_table(cruce_1_true,index=['serti_date','carrier_x']
                            ,columns=['status_code']
                            ,values=['importe','amount','transactionId','serti_transaction_id','comision','ivaComision',
                                     'comision_db','iva_comision_db']
                            ,aggfunc={'importe':np.sum,
                                      'amount':np.sum,
                                      'comision':np.sum,
                                      'ivaComision':np.sum,
                                      'comision_db':np.sum,
                                      'iva_comision_db':np.sum,
                                      'transactionId':len,
                                      'serti_transaction_id':len},margins=True).reset_index()
pivot_carrier.fillna(0,inplace=True)
pivot_carrier.head()

serti_date carrier_x     amount             comision  \
status_code                                         1        All         1   
0            2022-02-01 00:00:00       ATT  95,060.00  95,060.00  6,654.20   
1            2022-02-01 00:00:00  MOVISTAR  72,760.00  72,760.00  5,093.20   
2            2022-02-01 00:00:00    TELCEL 354,340.00 354,340.00 14,173.60   
3            2022-02-01 00:00:00    UNEFON  49,750.00  49,750.00  3,482.50   
4            2022-02-02 00:00:00       ATT  81,610.00  81,610.00  5,712.70   

                      comision_db              importe            ivaComision  \
status_code       All           1       All          1        All           1   
0            6,654.20    6,654.20  6,654.20  95,060.00  95,060.00    1,064.67   
1            5,093.20    5,093.20  5,093.20  72,760.00  72,760.00      814.91   
2           14,173.60   14,173.60 14,173.60 354,340.00 354,340.00    2,267.78   
3            3,482.50    3,482.50  3,482.50  49,750.00  49,750.00      557.20   
4            5,712.70    5,712.70  5,712.70  81,610.00  81,610.00      914.03   

                     iva_comision_db          serti_transaction_id        \
status_code      All               1      All                    1   All   
0           1,064.67        1,064.67 1,064.67                 1292  1292   
1             814.91          814.91   814.91                 1115  1115   
2           2,267.78        2,267.78 2,267.78                 5144  5144   
3             557.20          557.20   557.20                 1015  1015   
4             914.03          914.03   914.03                 1177  1177   

            transactionId        
status_code             1   All  
0                    1292  1292  
1                    1115  1115  
2                    5144  5144  
3                    1015  1015  
4                    1177  1177

In [53]:
pivot_carrier.shape

(113, 18)

### PIVOT RESUMEN DE COMISIONES

In [54]:
pivot_carrier_resumen = pd.pivot_table(cruce_1_true,index=['carrier_x','serti_date']
                            ,columns=['status_code']
                            ,values=['importe','amount','transactionId','serti_transaction_id','comision','ivaComision',
                                     'comision_db','iva_comision_db']
                            ,aggfunc={'importe':np.sum,
                                      'amount':np.sum,
                                      'comision':np.sum,
                                      'ivaComision':np.sum,
                                      'comision_db':np.sum,
                                      'iva_comision_db':np.sum,
                                      'transactionId':len,
                                      'serti_transaction_id':len})
pivot_carrier_resumen.head()

amount comision comision_db   importe ivaComision  \
status_code                  1        1           1         1           1   
carrier_x serti_date                                                        
ATT       2022-02-01 95,060.00 6,654.20    6,654.20 95,060.00    1,064.67   
          2022-02-02 81,610.00 5,712.70    5,712.70 81,610.00      914.03   
          2022-02-03 84,410.00 5,908.70    5,908.70 84,410.00      945.39   
          2022-02-04 86,250.00 6,037.50    6,037.50 86,250.00      966.00   
          2022-02-05 83,980.00 5,878.60    5,878.60 83,980.00      940.58   

                     iva_comision_db serti_transaction_id transactionId  
status_code                        1                    1             1  
carrier_x serti_date                                                     
ATT       2022-02-01        1,064.67                 1292          1292  
          2022-02-02          914.03                 1177          1177  
          2022-02-03          945.39                 1259          1259  
          2022-02-04          966.00                 1258          1258  
          2022-02-05          940.58                 1215          1215

In [55]:
pivot_carrier_resumen.shape

(112, 8)

# EXCEL FINAL

In [56]:
writer = pd.ExcelWriter('CONCILIADO FEBRERO COMISIONES 2022.xlsx', engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
pivot_serti.to_excel(writer, sheet_name='RESUMEN SERTI')
pivot_true.to_excel(writer, sheet_name='RESUMEN')
pivot_carrier.to_excel(writer, sheet_name='RESUMEN CARRIER')
pivot_carrier_resumen.to_excel(writer, sheet_name='RESUMEN COMISIONES')
cruce_1_true.to_excel(writer, sheet_name='DETALLE COMPLETO',index=False)
cruce_1_final.to_excel(writer, sheet_name='FOUND',index=False)
na_cruce_1_final.to_excel(writer, sheet_name='NOT FOUND',index=False)
duplicates_cruce_1_true.to_excel(writer, sheet_name='DUPLICADOS_INDEX_TOTAL',index=False)
writer.close()